Create a test sql database from titanic dataset.

https://python.langchain.com/docs/use_cases/sql/csv/

In [4]:
import pandas as pd
from pyprojroot import here

In [5]:
df = pd.read_csv(here("data/for_upload/titanic.csv"))
print(df.shape)
print(df.columns.tolist())
display(df.head(3))

(887, 8)
['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'Siblings/Spouses Aboard', 'Parents/Children Aboard', 'Fare']


,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250


### **SQL**

Using SQL to interact with CSV data is the recommended approach because it is easier to limit permissions and sanitize queries than with arbitrary Python.

Most SQL databases make it easy to load a CSV file in as a table (DuckDB, SQLite, etc.). Once you’ve done this you can use all of the chain and agent-creating techniques outlined in the SQL use case guide. Here’s a quick example of how we might do this with SQLite:

In [6]:
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine
db_path = str(here("data")) + "/test_sqldb.db"
db_path = f"sqlite:///{db_path}"

engine = create_engine(db_path)
# df.to_sql("titanic", engine, index=False)
df.to_sql("titanic", engine, index=False)

ValueError: Table 'titanic' already exists.

For multiple csv files, we can create a sql with multiple tables:
```
df1.to_sql("csv1_name", engine, index=False)
df2.to_sql("csv2_name", engine, index=False)
```

In [7]:
db = SQLDatabase(engine=engine)
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM titanic WHERE Age < 2;")

sqlite
['titanic']


"[(1, 2, 'Master. Alden Gates Caldwell', 'male', 0.83, 0, 2, 29.0), (0, 3, 'Master. Eino Viljami Panula', 'male', 1.0, 4, 1, 39.6875), (1, 3, 'Miss. Eleanor Ileen Johnson', 'female', 1.0, 1, 1, 11.1333), (1, 2, 'Master. Richard F Becker', 'male', 1.0, 2, 1, 39.0), (1, 1, 'Master. Hudson Trevor Allison', 'male', 0.92, 1, 2, 151.55), (1, 3, 'Miss. Maria Nakid', 'female', 1.0, 0, 2, 15.7417), (0, 3, 'Master. Sidney Leonard Goodwin', 'male', 1.0, 5, 2, 46.9), (1, 3, 'Miss. Helene Barbara Baclini', 'female', 0.75, 2, 1, 19.2583), (1, 3, 'Miss. Eugenie Baclini', 'female', 0.75, 2, 1, 19.2583), (1, 2, 'Master. Viljo Hamalainen', 'male', 0.67, 1, 1, 14.5), (1, 3, 'Master. Bertram Vere Dean', 'male', 1.0, 1, 2, 20.575), (1, 3, 'Master. Assad Alexander Thomas', 'male', 0.42, 0, 1, 8.5167), (1, 2, 'Master. Andre Mallet', 'male', 1.0, 0, 2, 37.0042), (1, 2, 'Master. George Sibley Richards', 'male', 0.83, 1, 1, 18.75)]"

**Equivalent in Pandas**

In [8]:
df[df["Age"]<2]

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
77,1,2,Master. Alden Gates Caldwell,male,0.83,0,2,29.0000
163,0,3,Master. Eino Viljami Panula,male,1.00,4,1,39.6875
171,1,3,Miss. Eleanor Ileen Johnson,female,1.00,1,1,11.1333
182,1,2,Master. Richard F Becker,male,1.00,2,1,39.0000
303,1,1,Master. Hudson Trevor Allison,male,0.92,1,2,151.5500
379,1,3,Miss. Maria Nakid,female,1.00,0,2,15.7417
384,0,3,Master. Sidney Leonard Goodwin,male,1.00,5,2,46.9000
466,1,3,Miss. Helene Barbara Baclini,female,0.75,2,1,19.2583
641,1,3,Miss. Eugenie Baclini,female,0.75,2,1,19.2583
751,1,2,Master. Viljo Hamalainen,male,0.67,1,1,14.5000


### **Create an agent to interact with the Database**

In [9]:
import os
from dotenv import load_dotenv
import warnings
warnings.filterwarnings("ignore")
print("Environment variables are loaded:", load_dotenv())
print("test by reading a variable:", os.getenv("OPENAI_API_TYPE"))

Environment variables are loaded: True
test by reading a variable: None


In [10]:
from langchain.chat_models import AzureChatOpenAI

model_name = os.getenv("gpt_deployment_name")
azure_openai_api_key = os.environ["OPENAI_API_KEY"]
azure_openai_endpoint = os.environ["OPENAI_API_BASE"]
llm = AzureChatOpenAI(
    openai_api_version=os.getenv("OPENAI_API_VERSION"),
    azure_deployment=model_name,
    model_name=model_name,
    temperature=0.0)

C:\Users\apoor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(


In [11]:
from langchain_community.agent_toolkits import create_sql_agent
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [12]:
agent_executor.invoke({"input": "Tell me more about Anders Johan Andersson"})



> Entering new SQL Agent Executor chain...


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse tool input: {'arguments': "# First, let's list the tables in the database\nfrom functions import sql_db_list_tables\n\ntables = sql_db_list_tables({})\nprint(tables)", 'name': 'python'} because the `arguments` is not valid JSON.

In [13]:
agent_executor.invoke({"input": "what's the average age of survivors"})



> Entering new SQL Agent Executor chain...


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse tool input: {'arguments': "# I will query the schema of the relevant tables to see what I can query\nfrom typing import List\nfrom functions import sql_db_schema\n\ndef get_table_schema(table_names: List[str]):\n    return sql_db_schema({\n        'table_names': ', '.join(table_names)\n    })\n\nget_table_schema(['passengers'])", 'name': 'python'} because the `arguments` is not valid JSON.

In [14]:
# Equivalence in Pandas
df[df["Survived"]==1]["Age"].mean()

28.408391812865496